## Import stuff

In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())

oversampler = RandomOverSampler(random_state=0)
undersampler = ClusterCentroids(random_state=0)


def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15139062358519593555
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


## Load training data from csv


In [2]:
features_df_normalized = pd.read_csv("train_features_normalized.csv")
labels_df = pd.read_csv("train_labels.csv")

X = np.array(features_df_normalized.iloc[:, 2:].values,  dtype=float).reshape(227940//12,35*12)


X_train = X[:int(X.shape[0]*0.8)]
X_valid = X[int(X.shape[0]*0.8):]


Y = np.array(labels_df.iloc[:, 1:].values,  dtype=float)
Y_train = Y[:int(Y.shape[0]*0.8)]
Y_valid = Y[int(Y.shape[0]*0.8):]


print("X shape: ", X.shape)
print("Y shape: ", Y.shape)


X shape:  (18995, 420)
Y shape:  (18995, 15)


## The function that trains and rates models

In [26]:
def find_best_model(col, X_train, Y_train, X_valid, Y_valid, X_test):
    models = []

    #task 1 and 2
    if(col<11):    
        for _ in range(0,1):    
            print("training 100+30 relu for col ", col)
            nn = tf.keras.Sequential([
                layers.Dense(100, activation='relu'),
                layers.Dense(30, activation='relu'),
                layers.Dense(1, activation='sigmoid')
            ])
            nn.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01))
            nn.fit(X_train, Y_train[:,col], epochs=15, verbose=False)
            models.append(nn)

            print("training 64+64 relu for col ", col)
            nn = tf.keras.Sequential([
                layers.Dense(64, activation='relu'),
                layers.Dense(64, activation='relu'),
                layers.Dense(1, activation='sigmoid')
            ])
            nn.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01))
            nn.fit(X_train, Y_train[:,col], epochs=15, verbose=False)
            models.append(nn)

        print("training rfc for col ", col)
        rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=10)
        rfc.fit(X_train, Y_train[:,col])
        models.append(rfc)

        print("training decision tree for col ", col)
        model = DecisionTreeClassifier(max_depth=5)
        model.fit(X_train, Y_train[:,col])
        models.append(model)
                        
        print("training Naive Bayes for col ", col)
        model = GaussianNB()
        model.fit(X_train, Y_train[:,col])
        models.append(model)
                        
        print("training QDA for col ", col)
        model = QuadraticDiscriminantAnalysis()
        model.fit(X_train, Y_train[:,col])
        models.append(model)

        print("training lr for col ", col)
        lr = LogisticRegression(max_iter=1000)
        lr.fit(X_train, Y_train[:,col])
        models.append(lr)

        print("training adaboost for col ", col)
        adaboost = AdaBoostClassifier(n_estimators=100, random_state=10)
        adaboost.fit(X_train, Y_train[:,col])
        models.append(adaboost)

        print("training ridge for col ", col)
        ridge = Ridge(alpha=1.0)
        ridge.fit(X_train, Y_train[:,col])
        models.append(ridge)

        print("training kernel ridge for col ", col)
        kernel_ridge = KernelRidge(alpha=1.0, degree=2)
        kernel_ridge.fit(X_train, Y_train[:,col])
        models.append(kernel_ridge)

        print("training linear kernel SVM for col ", col)
        svm_clf = SVC(kernel='linear', C=1.0, degree=2)
        svm_clf.fit(X_train, Y_train[:,col])
        models.append(svm_clf)

        print("training rbf kernel SVM for col ", col)
        rbf = svm.SVC(kernel="rbf")
        rbf.fit(X_train, Y_train[:,col])
        models.append(rbf)



        # these need too much ram
        # print("training poly kernel SVM for col ", col)
        # degree = 2
        # l2_coef = 0.01
        # model2 = make_pipeline(PolynomialFeatures(degree),Ridge(alpha=l2_coef, fit_intercept=False, solver="svd"))
        # model2.fit(X_train, Y_train[:,col])

        # print("training Gaussian Process for col ", col)
        # gp = GaussianProcessClassifier()#1.0 * RBF(1.0)
        # gp.fit(X_train, Y_train[:,col])
        # models.append(gp)

    
    #task 3
    if(col>=11):
        for _ in range(0,1):
            print("training 100+30 relu for col ", col)
            nn = tf.keras.Sequential([
            tf.keras.layers.Dense(units=100, activation='relu'),
            tf.keras.layers.Dense(units=30, activation='relu'),
            tf.keras.layers.Dense(units=1)
            ])
            nn.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.01))
            nn.fit(X_train, Y_train[:,col], epochs=30, verbose=False)
            models.append(nn)

            print("training 64+64 relu for col ", col)
            nn = tf.keras.Sequential([
            tf.keras.layers.Dense(units=64, activation='relu'),
            tf.keras.layers.Dense(units=64, activation='relu'),
            tf.keras.layers.Dense(units=1)
            ])
            nn.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.01))
            nn.fit(X_train, Y_train[:,col], epochs=30, verbose=False)
            models.append(nn)

    
    print("done training all models for col ", col)

    best_score = 0
    best_model = None
    for model in models:
        pred = model.predict(X_valid)
        if(col>=11):
            score = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y_valid[:,col], pred))
        else:
            score = metrics.roc_auc_score(Y_valid[:,col], pred)
        print("score for",type(model),": " , score)
        if score > best_score:
            best_score = score
            best_model = model

    print("best model: ", type(best_model))
    predictions = best_model.predict(X_test)
    print("predictions shape: ", predictions.shape)

    return predictions

## Writing results

In [27]:

X_test = pd.read_csv("test_features_normalized.csv")
X_test = np.array(X_test.iloc[:, 2:].values,  dtype=float).reshape(151968//12,35*12)

data = []

for i in range(0,15):
    pred = find_best_model(i, X[:int(X.shape[0]*0.8)], Y[:int(Y.shape[0]*0.8)], X[int(X.shape[0]*0.8):], Y[int(Y.shape[0]*0.8):], X_test)
    # pred = find_best_model(i, X[int(X.shape[0]*0.2):], Y[int(Y.shape[0]*0.2):], X[:int(X.shape[0]*0.2)], Y[:int(Y.shape[0]*0.2)], X_test)
    # print("pred shape: ", pred.shape)
    pred.reshape(pred.shape[0],1)
    data.append(pred)






# pred1 = []
# for i in range(0, 15):
#     Y_pred_lr = pred1.append(models[i].predict_proba(test_X_normalized)[:,1])


# pred1 = np.array(pred1).transpose()
# # pred1 = model1.predict(test_X_normalized)
# pred2 = model2.predict(test_X_normalized)
# # pred2 = model2.predict(test_X_small)
# # pred2 = (pred2+1)/2
# # np.clip(pred2, 0, 1, out=pred2)
# pred3 = model3.predict(test_X_normalized)

# print(pred2[pred2==0].shape)
# print(pred2[pred2>0.5].shape)


training 100+30 relu for col  0
training 64+64 relu for col  0
training 100+30 relu for col  0
training 64+64 relu for col  0
training 100+30 relu for col  0
training 64+64 relu for col  0
done training all models for col  0
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.7344099318806737
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.7346472441380073
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.742201166180758
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.7357484835504954
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.7373067304105123
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.7359809597501831
best model:  <class 'tensorflow.python.keras.engine.sequential.Sequential'>
predictions shape:  (12664, 1)
training 100+30 relu for col  1
training 64+64 relu for col  1
training 100+30 relu for col  1
trai

In [30]:
np_array = np.array(data).transpose()[0]
col_names = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2', 'LABEL_Sepsis', 'LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
print(len(col_names))
new_df = pd.DataFrame(np_array, columns=col_names)

output_csv = pd.read_csv("sample.csv")
output_csv.update(new_df)
output_csv.to_csv('submit.zip', index=False, float_format='%.3f', compression='zip')
output_csv.to_csv('submit.csv', index=False, float_format='%.3f')


15
